In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Ensure numbers are printed in a fixed decimal format
np.set_printoptions(suppress=True, precision=6)

# Define a simple unitary evolution (e.g., rotation around Y-axis)
def unitary_evolution(psi, theta):
    U = torch.tensor([[torch.cos(theta/2), -torch.sin(theta/2)], 
                      [torch.sin(theta/2),  torch.cos(theta/2)]], dtype=torch.float32)
    return U @ psi

# Step 1: Create a simple initial state |ψ₀⟩
psi_0_true = torch.tensor([1.0, 0.0], dtype=torch.float32)  # Choose a simple state |0⟩

# Step 2: Evolve to final state |ψ_f⟩
theta = torch.tensor(1.0)  # Rotation angle
psi_f = unitary_evolution(psi_0_true, theta)

# Print states before training
print("True initial state |ψ₀⟩:", np.array(psi_0_true))
print("Evolved final state |ψ_f⟩:", np.array(psi_f))

# Step 3: Reconstruct initial state using backpropagation
psi_0_guess = torch.randn(2, dtype=torch.float32, requires_grad=True)  # Start with a random guess
psi_0_guess.data = psi_0_guess.data / torch.norm(psi_0_guess.data)  # Normalize

optimizer = optim.Adam([psi_0_guess], lr=0.01)
loss_fn = nn.MSELoss()

# Training loop to find the best |ψ₀⟩
for epoch in range(300):
    optimizer.zero_grad()

    # Apply unitary evolution
    psi_pred = unitary_evolution(psi_0_guess, theta)  # Should match |ψ_f⟩

    # Compute loss
    loss = loss_fn(psi_pred, psi_f)

    # Backpropagate
    loss.backward()
    optimizer.step()

    # Renormalize |ψ₀⟩ after each step
    psi_0_guess.data = psi_0_guess.data / torch.norm(psi_0_guess.data)

    # Print psi_0_guess at each epoch without scientific notation
    #print(f"Epoch {epoch}, Loss: {loss.item():.6f}, |ψ₀_reconstructed⟩: {np.array(psi_0_guess.data)}")

# Step 4: Compare reconstructed |ψ₀⟩ with the true initial state
print("\nFinal reconstructed initial state |ψ₀_reconstructed⟩:", np.array(psi_0_guess.data))
print("True initial state |ψ₀_true⟩:", np.array(psi_0_true))


True initial state |ψ₀⟩: [1. 0.]
Evolved final state |ψ_f⟩: [0.877583 0.479426]

Final reconstructed initial state |ψ₀_reconstructed⟩: [1.       0.000054]
True initial state |ψ₀_true⟩: [1. 0.]


In [8]:
import torch
import torch.optim as optim

# Define a quantum rotation gate (RY rotation)
def quantum_circuit(theta):
    return torch.stack([torch.cos(theta / 2), torch.sin(theta / 2)])

# Loss function: Fidelity with target state |+⟩ = (|0⟩ + |1⟩)/√2
target_state = torch.tensor([0.707, 0.707], dtype=torch.float32)

# Initialize parameter
theta = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

optimizer = optim.Adam([theta], lr=0.1)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()

    # Compute quantum state
    psi = quantum_circuit(theta)

    # Compute loss (1 - fidelity)
    loss = 1 - torch.dot(psi, target_state).pow(2)

    # Backpropagate (automatic differentiation)
    loss.backward()  # ✅ No error now!
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}, Theta: {theta.item():.6f}")

# Final optimized parameter
print("Optimized Theta:", theta.item())


Epoch 0, Loss: 0.079543, Theta: 1.100000
Epoch 10, Loss: 0.010248, Theta: 1.786991
Epoch 20, Loss: 0.000302, Theta: 1.538400
Epoch 30, Loss: 0.001394, Theta: 1.522803
Epoch 40, Loss: 0.001029, Theta: 1.620440
Epoch 50, Loss: 0.000488, Theta: 1.541331
Epoch 60, Loss: 0.000326, Theta: 1.584196
Epoch 70, Loss: 0.000303, Theta: 1.565792
Epoch 80, Loss: 0.000302, Theta: 1.572331
Epoch 90, Loss: 0.000302, Theta: 1.570414
Optimized Theta: 1.5701297521591187


In [15]:
import torch
import torch.optim as optim
import numpy as np

# Define Hamiltonians H0 (initial) and Hf (final)
H0 = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64, requires_grad=False)  # Pauli Z
Hf = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64, requires_grad=False)  # Pauli X

# Time evolution function using matrix exponential
def time_evolution(H, dt):
    U = torch.linalg.matrix_exp(-1j * H * dt)  # Unitary evolution step
    return U / torch.linalg.norm(U)  # Ensure unitary scaling

# Define adiabatic evolution
def hamiltonian_evolution(psi, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.clone()
    for step in range(steps):
        s = step / steps  # Interpolation factor
        H = (1 - s) * H0 + s * Hf  # Time-dependent Hamiltonian
        U = time_evolution(H, dt)  # Compute unitary evolution
        psi_t = torch.mm(U, psi_t.unsqueeze(1)).squeeze(1)  # Matrix multiplication
    return psi_t / torch.norm(psi_t)  # Normalize final state

# Initialize trainable complex state
psi_0_real = torch.tensor([1.0, 0.0], dtype=torch.float32, requires_grad=True)
psi_0_imag = torch.tensor([0.0, 1.0], dtype=torch.float32, requires_grad=True)

def get_psi_0():
    psi = torch.complex(psi_0_real, psi_0_imag)
    return psi / torch.norm(psi)  # Ensure normalization

# Target final state
psi_f_target = torch.tensor([0.707, 0.707], dtype=torch.complex64, requires_grad=False)

# Optimizer & Learning Rate Scheduling
optimizer = optim.AdamW([psi_0_real, psi_0_imag], lr=0.1, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.7)  # Decay LR

# **New Fidelity Loss Function (Maximizing Inner Product)**
def fidelity_loss(psi_final, psi_target):
    psi_final = psi_final / torch.norm(psi_final)  # Normalize
    psi_target = psi_target / torch.norm(psi_target)  # Normalize
    return -torch.abs(torch.vdot(psi_final, psi_target))  # ✅ Maximize overlap instead of minimizing loss

# Training loop
for epoch in range(1000):  # Extended training to ensure convergence
    optimizer.zero_grad()
    
    # Construct initial state
    psi_0 = get_psi_0()

    # Adiabatic evolution
    psi_final = hamiltonian_evolution(psi_0)

    # Compute fidelity loss (maximize overlap)
    loss = fidelity_loss(psi_final, psi_f_target)

    # Backpropagation
    loss.backward()
    
    # Gradient Clipping (Prevents Small Updates)
    torch.nn.utils.clip_grad_norm_([psi_0_real, psi_0_imag], max_norm=1.0)

    optimizer.step()
    scheduler.step()  # Adjust learning rate over time

    # Print progress
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}, Learning Rate: {scheduler.get_last_lr()[0]:.5f}, |ψ₀_reconstructed⟩: {get_psi_0().detach().numpy()}")

# Final results
final_psi_0 = get_psi_0().detach().numpy()
print("\nFinal reconstructed initial state |ψ₀_reconstructed⟩:", final_psi_0)
print("Expected final state |ψ_f_target⟩:", psi_f_target.numpy())


Epoch 0, Loss: -0.294580, Learning Rate: 0.10000, |ψ₀_reconstructed⟩: [0.770155+0.070015j 0.070015+0.630125j]
Epoch 50, Loss: -0.999175, Learning Rate: 0.10000, |ψ₀_reconstructed⟩: [0.626262+0.458098j 0.603353-0.18414j ]
Epoch 100, Loss: -0.999995, Learning Rate: 0.10000, |ψ₀_reconstructed⟩: [0.600036+0.469041j 0.614218-0.206622j]
Epoch 150, Loss: -1.000000, Learning Rate: 0.10000, |ψ₀_reconstructed⟩: [0.597739+0.46932j  0.615804-0.207922j]
Epoch 200, Loss: -1.000000, Learning Rate: 0.07000, |ψ₀_reconstructed⟩: [0.597816+0.469262j 0.615799-0.207849j]
Epoch 250, Loss: -1.000000, Learning Rate: 0.07000, |ψ₀_reconstructed⟩: [0.59782 +0.469259j 0.615799-0.207844j]
Epoch 300, Loss: -1.000000, Learning Rate: 0.07000, |ψ₀_reconstructed⟩: [0.597818+0.469259j 0.6158  -0.207844j]
Epoch 350, Loss: -1.000000, Learning Rate: 0.07000, |ψ₀_reconstructed⟩: [0.597818+0.469259j 0.6158  -0.207844j]
Epoch 400, Loss: -1.000000, Learning Rate: 0.04900, |ψ₀_reconstructed⟩: [0.597818+0.46926j  0.6158  -0.2078

In [20]:
import numpy as np
from scipy.linalg import expm
import torch

# Define Hamiltonians
H0 = np.array([[1, 0], [0, -1]], dtype=np.complex64)  # Pauli Z
Hf = np.array([[0, 1], [1, 0]], dtype=np.complex64)  # Pauli X

# Time evolution function
def time_evolution(H, dt):
    return expm(-1j * H * dt)  # Unitary evolution

# Adiabatic evolution function
def hamiltonian_evolution(psi, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H0 + s * Hf
        U = time_evolution(H, dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# 🔹 Step 1: Start with a known initial state |ψ₀⟩
true_psi_0 = np.array([1.0, 0.0], dtype=np.complex64)  # Known initial state |0⟩

# 🔹 Step 2: Evolve forward to get final state |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0)

# Define the Oracle: Flips phase of correct initial states
def oracle(psi_0_guess, psi_f_target, threshold=0.99):
    psi_f_guess = hamiltonian_evolution(psi_0_guess)
    fidelity = np.abs(np.vdot(psi_f_guess, psi_f_target))**2
    if fidelity >= threshold:
        return -psi_0_guess  # Flip phase if correct
    return psi_0_guess  # Otherwise, leave unchanged

# Define the Diffusion Operator (Grover's Amplification)
def diffusion_operator(psi_super):
    mean = np.mean(psi_super)
    return 2 * mean - psi_super

# 🔹 Step 3: Run Backpropagation (Grover-style search)
# Create a set of possible initial states (superposition-like approach)
psi_candidates = np.array([
    [np.cos(theta/2), np.sin(theta/2)] for theta in np.linspace(0, np.pi, 100)
], dtype=np.complex64)

# Apply Grover Iterations
num_iterations = int(np.sqrt(len(psi_candidates)))
for _ in range(num_iterations):
    psi_candidates = np.array([oracle(psi, psi_f) for psi in psi_candidates])  # Apply Oracle
    psi_candidates = diffusion_operator(psi_candidates)  # Apply Diffusion Operator

# Find the best candidate
best_psi_0 = max(psi_candidates, key=lambda psi: np.abs(np.vdot(hamiltonian_evolution(psi), psi_f))**2)

# 🔹 Step 4: Compare Results
print("\nTrue initial state |ψ₀_true⟩:", true_psi_0)
print("Evolved final state |ψ_f⟩:", psi_f)
print("Reconstructed initial state |ψ₀_reconstructed⟩:", best_psi_0)
print("\nFidelity with true |ψ₀⟩:", np.abs(np.vdot(best_psi_0, true_psi_0))**2)



True initial state |ψ₀_true⟩: [1.+0.j 0.+0.j]
Evolved final state |ψ_f⟩: [ 0.76282 -0.452789j -0.156136-0.434407j]
Reconstructed initial state |ψ₀_reconstructed⟩: [-1.+0.j  0.+0.j]

Fidelity with true |ψ₀⟩: 0.9999995231628986


In [36]:
import numpy as np
from scipy.linalg import expm

# Number of qubits
n_qubits = 4
dim = 2 ** n_qubits  # Hilbert space dimension

# Define Pauli matrices
I = np.eye(2, dtype=np.complex64)
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)

# Build multi-qubit Hamiltonians (Pauli-Z and Pauli-X sums)
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Time evolution function
def time_evolution(H, dt):
    return expm(-1j * H * dt)

# Adiabatic evolution function
def hamiltonian_evolution(psi, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H0 + s * Hf
        U = time_evolution(H, dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)

# 🔹 Step 1: Start with a known initial 4-qubit state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[[0, 3, 5]] = 1 / np.sqrt(3)  # Example: A superposition state

# 🔹 Step 2: Evolve forward to get final state |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0)

# 🔹 Step 3: Define a Smooth Oracle Function
def oracle(psi_0_guess, psi_f_target, weight=0.9):
    """Instead of flipping phase, smoothly adjust states based on fidelity"""
    psi_f_guess = hamiltonian_evolution(psi_0_guess)
    fidelity = np.abs(np.vdot(psi_f_guess, psi_f_target))**2
    return np.exp(1j * weight * fidelity) * psi_0_guess  # Smooth phase shift

# 🔹 Step 4: Apply a Corrected Diffusion Operator
def diffusion_operator(psi_super):
    """Applies full-space diffusion"""
    psi_avg = np.mean(psi_super, axis=0)
    return 2 * psi_avg - psi_super

# 🔹 Step 5: Use a Superposition of All Basis States as Candidates
basis_states = np.eye(dim, dtype=np.complex64)  # Full 16 basis states

# Apply Grover Iterations
num_iterations = int(np.sqrt(dim))  # ~4 iterations for 16 states
for _ in range(num_iterations):
    basis_states = np.array([oracle(psi, psi_f) for psi in basis_states])  # Apply Oracle
    basis_states = np.array([diffusion_operator(psi) for psi in basis_states])  # Apply Diffusion

# Find the best candidate
best_psi_0 = max(basis_states, key=lambda psi: np.abs(np.vdot(hamiltonian_evolution(psi), psi_f))**2)

# 🔹 Step 6: Compare Results
print("\nTrue initial state |ψ₀_true⟩:", true_psi_0)
print("Evolved final state |ψ_f⟩:", psi_f)
print("Reconstructed initial state |ψ₀_reconstructed⟩:", best_psi_0)

# Compute Fidelity with the original initial state
fidelity = np.abs(np.vdot(best_psi_0, true_psi_0) / (np.linalg.norm(best_psi_0) * np.linalg.norm(true_psi_0)))**2
print("\nFidelity with true |ψ₀⟩:", fidelity)



True initial state |ψ₀_true⟩: [0.57735+0.j 0.     +0.j 0.     +0.j 0.57735+0.j 0.     +0.j 0.57735+0.j
 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j
 0.     +0.j 0.     +0.j 0.     +0.j 0.     +0.j]
Evolved final state |ψ_f⟩: [-0.373223-0.228572j -0.243183-0.296112j -0.198386-0.063976j
  0.279047+0.095057j -0.198386-0.063976j  0.279047+0.095057j
 -0.175276+0.095057j  0.120881-0.365342j -0.153589+0.16816j
 -0.175276+0.095057j -0.05225 +0.095057j  0.076084-0.133206j
 -0.05225 +0.095057j  0.076084-0.133206j  0.031287+0.09893j
 -0.174752-0.097787j]
Reconstructed initial state |ψ₀_reconstructed⟩: [-0.      -0.j       -0.      -0.j       -0.      -0.j
 -0.      -0.j       -0.      -0.j        0.671641+0.740877j
 -0.      -0.j       -0.      -0.j       -0.      -0.j
 -0.      -0.j       -0.      -0.j       -0.      -0.j
 -0.      -0.j       -0.      -0.j       -0.      -0.j
 -0.      -0.j      ]

Fidelity with true |ψ₀⟩: 0.33333345901888833


In [49]:
import numpy as np
from scipy.linalg import expm

# Number of qubits
n_qubits = 4
dim = 2 ** n_qubits  # Hilbert space dimension

# Define identity and Pauli matrices
I = np.eye(2, dtype=np.complex64)
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)

# Build multi-qubit Hamiltonians (Pauli-Z and Pauli-X sums)
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Time evolution function
def time_evolution(H, dt):
    return expm(-1j * H * dt)

# Adiabatic evolution function
def hamiltonian_evolution(psi, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H0 + s * Hf
        U = time_evolution(H, dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# 🔹 Step 1: Start with a known initial 4-qubit state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[5] = 1

# 🔹 Step 2: Evolve forward to get final state |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0)

# 🔹 Step 3: Add noise to one qubit (Qubit 2)
def add_noise_to_qubit(psi, qubit_index, noise_strength=0.1):
    """Applies a bit-flip (Pauli-X) noise to a single qubit with some probability"""
    noise_matrix = np.kron(np.kron(np.eye(2**qubit_index), X), np.eye(2**(n_qubits-qubit_index-1)))
    psi_noisy = (1 - noise_strength) * psi + noise_strength * (noise_matrix @ psi)
    return psi_noisy / np.linalg.norm(psi_noisy)  # Normalize

# Apply noise to qubit 2
psi_f_noisy = add_noise_to_qubit(psi_f, qubit_index=2, noise_strength=0.2)

# Define the Oracle: Flips phase of correct initial states
def oracle(psi_0_guess, psi_f_target, threshold=0.85):
    psi_f_guess = hamiltonian_evolution(psi_0_guess)
    fidelity = np.abs(np.vdot(psi_f_guess, psi_f_target))**2
    return -psi_0_guess if fidelity >= threshold else psi_0_guess  # Flip phase if correct

Define Grover's Diffusion Operator
def diffusion_operator(psi_super):
    """Grover’s diffusion operator for state amplification"""
    U_s = 2 * np.outer(psi_super, psi_super.conj()) - np.eye(dim)
    return U_s @ psi_super

# 🔹 Step 4: Use all computational basis states as candidates
basis_states = np.eye(dim, dtype=np.complex64)  # 16 basis states for 4 qubits

# Apply Grover Iterations
num_iterations = int(np.sqrt(dim))  # ~4 iterations for 16 states
for _ in range(num_iterations):
    basis_states = np.array([oracle(psi, psi_f_noisy) for psi in basis_states])  # Apply Oracle
    basis_states = np.array([diffusion_operator(psi) for psi in basis_states])  # Apply Diffusion

# Find the best candidate
best_psi_0 = max(basis_states, key=lambda psi: np.abs(np.vdot(hamiltonian_evolution(psi), psi_f_noisy))**2)

# 🔹 Step 5: Compare Results
print("\nTrue initial state |ψ₀_true⟩:", true_psi_0)
print("Evolved final state |ψ_f⟩:", psi_f)
print("Noisy final state |ψ_f_noisy⟩:", psi_f_noisy)
print("Reconstructed initial state |ψ₀_reconstructed⟩:", best_psi_0)

# Compute Fidelity with the original initial state
fidelity = np.abs(np.vdot(best_psi_0, true_psi_0) / (np.linalg.norm(best_psi_0) * np.linalg.norm(true_psi_0)))**2
print("\nFidelity with true |ψ₀⟩:", fidelity)



True initial state |ψ₀_true⟩: [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Evolved final state |ψ_f⟩: [-0.155641+0.062394j -0.061057-0.316395j  0.016534+0.085677j
 -0.167682-0.j       -0.061057-0.316395j  0.61923 -0.j
 -0.167682-0.j        0.061057-0.316395j  0.016534+0.085677j
 -0.167682-0.j        0.045407+0.j       -0.016534+0.085677j
 -0.167682+0.j        0.061057-0.316395j -0.016534+0.085677j
 -0.155641-0.062394j]
Noisy final state |ψ_f_noisy⟩: [-0.141894+0.078495j -0.096444-0.296319j -0.020957+0.094849j
 -0.171338-0.07408j  -0.096444-0.296319j  0.594235-0.07408j
 -0.171338-0.07408j   0.202168-0.296319j  0.026116+0.08024j
 -0.160913+0.02006j   0.046397+0.02006j  -0.054745+0.08024j
 -0.160913+0.02006j   0.020742-0.310928j -0.054745+0.08024j
 -0.131469-0.132515j]
Reconstructed initial state |ψ₀_reconstructed⟩: [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -1.-0.j -0.-0.j -0.-0.j -0.-0.j
 -0.-0.j -0.-0.j -0.-0.j -0.-0.

In [121]:
import numpy as np
from scipy.linalg import expm

# Number of qubits
n_qubits = 4
i = 1 # bitflip index for psi_0
j = 2 # bitflip index for psi_0
dim = 2 ** n_qubits  # Hilbert space dimension
noise = 1.0

# Define identity and Pauli matrices
I = np.eye(2, dtype=np.complex64)
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)

# Build multi-qubit Hamiltonians (Pauli-Z and Pauli-X sums)
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Time evolution function
def time_evolution(H, dt):
    return expm(-1j * H * dt)

# Adiabatic evolution function
def hamiltonian_evolution(psi, H_init, H_final, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H_init + s * H_final
        U = time_evolution(H, dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# Start with a known initial qubit state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[i] = 1
true_psi_0[i+1] = 1

print(true_psi_0)

# Evolve forward to get final state |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0, H0, Hf)

def apply_kraus_to_density_matrix(rho, kraus_ops):
    return sum(K @ rho @ K.conj().T for K in kraus_ops)

# Define noise (Depolarizing Channel)
def depolarizing_channel(p):
    K0 = np.sqrt(1 - (3 * p / 4)) * I
    K1 = np.sqrt(p / 4) * X
    K2 = np.sqrt(p / 4) * Y
    K3 = np.sqrt(p / 4) * Z

    return [K0, K1, K2, K3]

def apply_kraus_to_multi_qubit_density_matrix(rho, kraus_ops, qubit_index, n_qubits):
    dim = 2 ** n_qubits  # Full Hilbert space dimension
    rho_new = np.zeros((dim, dim), dtype=complex)

    for K in kraus_ops:
        # Expand Kraus operator to the full Hilbert space
        I_before = np.eye(2 ** qubit_index, dtype=complex)
        I_after = np.eye(2 ** (n_qubits - qubit_index - 1), dtype=complex)
        K_full = np.kron(np.kron(I_before, K), I_after)  # Expand to full space

        # Apply the expanded Kraus operator to the density matrix
        rho_new += K_full @ rho @ K_full.conj().T

    return rho_new

# Define the depolarizing channel for a single qubit
kraus_ops = depolarizing_channel(noise)  # Depolarizing channel

# Apply noise to a specific qubit (e.g., qubit 2 in a 5-qubit system)
rho_0 = np.outer(psi_f, psi_f.conj())
rho_noisy = apply_kraus_to_multi_qubit_density_matrix(rho_0, kraus_ops, j, n_qubits)

Hn = Hf + rho_noisy
# Evolve forward to get final state |ψ_f⟩
psi_f_noisy = hamiltonian_evolution(true_psi_0, H0, Hn)
#psi_f_noisy = psi_f

noise_fidelity = np.abs(np.vdot(psi_f_noisy, psi_f))**2

# Step 4: Define Grover search components

# Define computational basis states
basis_states = np.eye(dim, dtype=np.complex64)

# Define uniform superposition state |s⟩
psi_s = np.ones(dim, dtype=np.complex64) / np.sqrt(dim)

# Construct Diffusion Operator: D = 2|s⟩⟨s| - I
diffusion = 2 * np.outer(psi_s, psi_s.conj()) - np.eye(dim)

# Modify Oracle to Evolve Backwards Instead of Knowing Winning State

def backward_evolution(psi, T=1.0, steps=50):
    """Evolves the state backward using the reversed adiabatic process."""
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * Hf + s * H0  # Reverse Hamiltonian interpolation
        U = time_evolution(H, -dt)  # Reverse time evolution
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# Estimates |ψ₀⟩ by evolving |ψ_f_noisy⟩ backward
estimated_psi_0 = backward_evolution(psi_f_noisy)

# Generate new oracle using backward evolution
oracle = np.eye(dim) - 2 * np.outer(estimated_psi_0, estimated_psi_0.conj())

# Number of Grover Iterations (~π/4 * sqrt(N))
num_iterations = int(np.floor(np.pi / 4 * np.sqrt(dim)))

# Initialize search state in |s⟩ (equal superposition of all states)
psi_search = psi_s.copy()

# Apply Grover iterations
for i in range(num_iterations):
    psi_search = oracle @ psi_search  # Apply Oracle
    psi_search = diffusion @ psi_search  # Apply Diffusion
    print(f"Iteration {i+1}: {psi_search}")

# Measure the final state by finding the largest amplitude
winning_index = np.argmax(np.abs(psi_search)**2)
winning_state = np.zeros(dim, dtype=np.complex64)
winning_state[winning_index] = 1

# Compute Fidelity with true |ψ₀⟩
fidelity = np.abs(np.vdot(winning_state, true_psi_0))**2

# Print results
print("\n# Step 5: Compare Results")
print("\nTrue initial state |ψ₀_true⟩:\n", true_psi_0)
print("\nEvolved final state |ψ_f⟩:\n", psi_f)
print("\nNoisy final state |ψ_f_noisy⟩:\n", psi_f_noisy)
print("Fidelity |ψ^noise_f⟩ with true |ψ_f⟩:", noise_fidelity)
print("\nEstimate |ψ₀⟩ :\n", estimated_psi_0)
print("\nReconstructed initial state |ψ₀_reconstructed⟩:\n", winning_state)
print("\nFidelity with true |ψ₀⟩:", fidelity)

[0.+0.j 1.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Iteration 1: [0.110889+0.012515j 0.621935-0.036485j 0.621935-0.036485j
 0.142877+0.010162j 0.125831-0.000405j 0.124642+0.011545j
 0.124642+0.011545j 0.12574 +0.00034j  0.125831-0.000405j
 0.124642+0.011545j 0.124642+0.011545j 0.12574 +0.00034j
 0.125922-0.00001j  0.128522+0.002065j 0.128522+0.002065j
 0.126082+0.000123j]
Iteration 2: [-0.083201+0.018825j  0.685512-0.054881j  0.685512-0.054881j
 -0.035084+0.015285j -0.060726-0.000609j -0.062514+0.017365j
 -0.062514+0.017365j -0.060863+0.000512j -0.060726-0.000609j
 -0.062514+0.017365j -0.062514+0.017365j -0.060863+0.000512j
 -0.060588-0.000015j -0.056677+0.003107j -0.056677+0.003107j
 -0.060348+0.000185j]
Iteration 3: [-0.23604 +0.015802j  0.40921 -0.046066j  0.40921 -0.046066j
 -0.195651+0.01283j  -0.217175-0.000511j -0.218676+0.014576j
 -0.218676+0.014576j -0.217289+0.00043j  -0.217175-0.000511j
 -0.218676+0.014576j -0.21